# JPEG Compression

- Valérian FAYT <valerian.fayt>
- Romain HERMARY <romain.hermary>
- Quentin LE HELLOCO <quentin.le-helloco>

## Project

#### In this notebook, you have to:
* Illustrate and comment every different step of your algorithm, like if you had to explain it to someone who never heard of JPEG.
* Implement and analyses all relevant tests to demonstrate the proper functioning of the algorithm.

#### Your JPEG code should at least:
* Manage color, leaving the choice to the user to compress in RGB or YUV, as well as the sub-sampling options (4:4:4, 4:2:2 and 4:2:0) of chrominance.
* Manage images whose dimensions are not 8 multiples
* Let the user choose the quality indicator q for the luminance quantification matrix.
* Return for each macro-block, a compression indicator. This indicator may be define as in the course (number of coefficient by macro-block to code without compression (64) divide by the number of coefficient not null after zigzag linearisation of the DCT quantified matrix). It can be any other relevant indicator as well, if the choice is justified.

#### Bonus:
* Implement the conversion of DCT coefficients after quantification by Huffman table.

### JPEG Algorithm

* Step 1 :
Split images in 8x8 blocks (if not multiple, add new columns/row with symmetrical inputs.

* Step 2 :
Apply DCT matrix (don't forget to -128)

* Step 3 :
Quantification of the matrix

* Step 4 :
Zigzag + Huffman compression

(Reversible to uncompress JPEG as well)

### Import

In [ ]:
import numpy as np
from PIL import Image

### Get list of pixel from Image

In [ ]:
im = Image.open('Images/amidala_crop.png', 'r')
im = im.convert("L")
im

In [ ]:
# data = np.asarray(im)
#you can create images from array like that (rot90) for this one ->
#rotate = Image.fromarray(np.rot90(data), 'L')

### Image Split

If the image is not a multiple of 8, make it one by adding row and cells.
Divide the image into macro-block of 8x8pixels.

In [ ]:
def split(data):
    """
    Split a multiple of 8 image into 8x8 macro block list
    """
    
    blocks = []
    
    for i in range(0, data.shape[0], 8):
        for j in range(0, data.shape[1], 8):
            
            block = [[], [], [], [], [], [], [], []]
            
            for a in range (i, i + 8):
                for b in range (j, j + 8):
                    block[a - i].append(data[a][b])
                    
            blocks.append(block)
    return blocks

In [ ]:
def multiple_8(matrix):
    """
    Make any dimension image into a multiple of 8 by adding rows and columns
    by border pixels mirroring
    """
    
    a, b = matrix.shape
    print('Dimension:', matrix.shape)

    if a % 8 != 0 or b % 8 != 0 :
        print("Need padding...")
    else:
        return matrix

    padding_a = 8 - a % 8
    padding_b = 8 - b % 8

    # If the number of pixels to add on one line or a column, it is problematic
    print("Padding:", padding_a, padding_b)
    if padding_a > a or padding_b > b:
        print("Error: cannot do mirror padding, for now")
        return matrix
    
    matrix = matrix.tolist()
    
    # Mirror the pixels on the lines by copying and reversing the pixel line
    for i in range(a):
        tmp = matrix[i][b - padding_b:].copy()
        tmp.reverse()
        matrix[i] += tmp
    
    # Mirror the border lines starting from the bottom
    for i in range(padding_a):
        matrix.append(matrix[a - 1 - i])
    
    matrix = np.array(matrix)
    print('New Dimension:', matrix.shape)

    return matrix.astype('uint8')

In [ ]:
test = np.arange(1, 127)
test = test.reshape(9, 14)

test = multiple_8(test)
print(test)

In [ ]:
# Load the image into a numpy matrix
data = np.asarray(im)

# Make the matrix dimensions multiples of 8
data = multiple_8(data)
img = Image.fromarray(data, 'L')
display(img)

# Cut into macro-block
macro_blocks = split(data)

In [ ]:
# See a macro-block
len(macro_blocks)
block = macro_blocks[3479]
test_block = np.array(block)
test_macro = Image.fromarray(test_block, 'L')

test_macro = test_macro.resize((200,200), resample=0)
display(test_macro)
block

### Apply DCT matrix

explain here

### Quantification of the matrix

Explain here

### Zigzag + Huffman

Explain here

In [ ]:
def zigzag(matrix):
    x, y = 0, 0
    a, b = 7, 7
    direction = (0, 0)
    res = []
    
    while True:
        res.append(matrix[x][y])
        if y == b:
            res.append(matrix[x + 1][y])
            y -= 1
            x += 2
            direction = (1, -1)
        elif x == 0:
            res.append(matrix[x][y + 1])
            x +=1
            direction = (1, -1)
        elif x == a:
            res.append(matrix[x][y + 1])
            if y + 1 == b:
                break
            y += 2
            x -= 1
            direction = (-1, 1)
        elif y == 0:
            res.append(matrix[x + 1][y])
            y += 1
            direction = (-1, 1)
        else:
            x += direction[0]
            y += direction[1]

    return res

In [ ]:
test = np.arange(1, 65)
test = test.reshape(8, 8)

print(test)

zz = zigzag(test)
print(zz)

### RGB - YUV 
Explain here